In [1]:
import csv
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import shapefile
import pickle
import os

In [2]:
#read link data
link_path = os.path.join("transportation_data","wislr-data","wislr_rdwy_link.csv")
link_data = pd.read_csv(link_path)

#read link time
#time_path = os.path.join("link_time.csv")
time_path = os.path.join("link_time_updated.csv")
time_data = pd.read_csv(time_path,index_col=0)

#read link name
name_path = os.path.join("transportation_data","wislr-data","wislr_rdwy_rte_link.csv")
name_data = pd.read_csv(name_path)

In [3]:
RDWY_LINK_ID = []
# add road name and link id
REF_SITE_FROM_ID = list(time_data["FROM"])
REF_SITE_TO_ID = list(time_data["TO"])

for i in range(len(REF_SITE_FROM_ID)):
    RDWY_LINK_ID.append(link_data.loc[(link_data["REF_SITE_FROM_ID"] == int(REF_SITE_FROM_ID[i])) & \
              (link_data["REF_SITE_TO_ID"] == int(REF_SITE_TO_ID[i])),"RDWY_LINK_ID"].iloc[0])

time_data['RDWY_LINK_ID'] = RDWY_LINK_ID


In [4]:
# add roadway rte id
RDWY_RTE_ID = []
for i in list(time_data['RDWY_LINK_ID']):
    RDWY_RTE_ID.append(name_data.loc[name_data['RDWY_LINK_ID'] == i,'RDWY_RTE_ID'].iloc[0])

time_data["RDWY_RTE_ID"] = RDWY_RTE_ID

In [5]:
time_data['FROM'] = time_data['FROM'].astype(str)
time_data['TO'] = time_data['TO'].astype(str)

In [6]:
time_data

,FROM,TO,TIME,RDWY_LINK_ID,RDWY_RTE_ID
0,1661200,1660820,122.295420,4818580,400333
1,1661678,1661705,5.765192,4819008,396810
2,1661704,1661705,9.632979,4819009,396788
3,1661723,1661697,21.199852,4819014,396769
4,1661705,1661723,9.632979,4819015,396736
...,...,...,...,...,...
3127,1663210,1663276,34.673847,5571520,400483
3128,1661558,1661551,11.566873,5571521,400548
3129,1661551,1661558,11.566873,5571522,400548
3130,2016715,2005025,38.303711,5571523,400488


In [7]:
name_data.head()

,WISLR_PTY_ID,WISLR_PTY_TYCD,WISLR_PTY_NM,RDWY_RTE_ID,SORT_NM,RDWY_LINK_ID,RWLK_STRT_OSET,RWLK_END_OSET,RWRL_SORT,BASC_RTE_FL,DIR_INDC
0,333,T,CROSS PLAINS,394349,AIRPORT RD,4840537,264,528,100,Y,>
1,333,T,CROSS PLAINS,394349,AIRPORT RD,5253200,0,132,200,Y,>
2,333,T,CROSS PLAINS,394349,AIRPORT RD,5253202,197,7762,300,Y,>
3,333,T,CROSS PLAINS,394349,AIRPORT RD,5253201,0,7565,400,N,<
4,333,T,CROSS PLAINS,394349,AIRPORT RD,5253199,185,317,500,N,<


In [8]:
name_data

,WISLR_PTY_ID,WISLR_PTY_TYCD,WISLR_PTY_NM,RDWY_RTE_ID,SORT_NM,RDWY_LINK_ID,RWLK_STRT_OSET,RWLK_END_OSET,RWRL_SORT,BASC_RTE_FL,DIR_INDC
0,333,T,CROSS PLAINS,394349,AIRPORT RD,4840537,264,528,100,Y,>
1,333,T,CROSS PLAINS,394349,AIRPORT RD,5253200,0,132,200,Y,>
2,333,T,CROSS PLAINS,394349,AIRPORT RD,5253202,197,7762,300,Y,>
3,333,T,CROSS PLAINS,394349,AIRPORT RD,5253201,0,7565,400,N,<
4,333,T,CROSS PLAINS,394349,AIRPORT RD,5253199,185,317,500,N,<
...,...,...,...,...,...,...,...,...,...,...,...
3376,379,C,MIDDLETON,702113,BREAKERS ROCK RD,5422167,0,947,400,N,<
3377,379,C,MIDDLETON,744117,CROSSOVER USH 00012,5519750,0,72,100,Y,>
3378,379,C,MIDDLETON,744117,CROSSOVER USH 00012,5519749,0,72,200,N,<
3379,379,C,MIDDLETON,756816,SCHWARTZ RD,5035654,0,898,99,Y,>


In [9]:
time_data.head()

,FROM,TO,TIME,RDWY_LINK_ID,RDWY_RTE_ID
0,1661200,1660820,122.295420,4818580,400333
1,1661678,1661705,5.765192,4819008,396810
2,1661704,1661705,9.632979,4819009,396788
3,1661723,1661697,21.199852,4819014,396769
4,1661705,1661723,9.632979,4819015,396736


In [10]:
arc_data = time_data.copy()
arc_data

,FROM,TO,TIME,RDWY_LINK_ID,RDWY_RTE_ID
0,1661200,1660820,122.295420,4818580,400333
1,1661678,1661705,5.765192,4819008,396810
2,1661704,1661705,9.632979,4819009,396788
3,1661723,1661697,21.199852,4819014,396769
4,1661705,1661723,9.632979,4819015,396736
...,...,...,...,...,...
3127,1663210,1663276,34.673847,5571520,400483
3128,1661558,1661551,11.566873,5571521,400548
3129,1661551,1661558,11.566873,5571522,400548
3130,2016715,2005025,38.303711,5571523,400488


In [11]:
arc_data = time_data.copy()
arc_data['use'] = 1
arc_data.loc[arc_data["FROM"]==arc_data["TO"] ,'use'] = 0
arc_data['turn_rteids'] = 0
#change data type to string
arc_data['FROM'] = arc_data['FROM'].astype(str)
arc_data['TO'] = arc_data['TO'].astype(str)
# start processing nodes

# find all the unique nodes
nodes = set(list(arc_data["FROM"]) + list(arc_data["TO"]))
interesction_index = 1

loopTrack = 1
# for each node, start generating buffer node acccording to the in and out arc
for N in nodes:
    new_arcs = []
    out_N = list(arc_data[(arc_data['FROM'] == N)&(arc_data['use'] == 1)]['TO'])
    out_N_time = list(arc_data[(arc_data['FROM'] == N)&(arc_data['use'] == 1)]['TIME'])
    out_N_id = list(arc_data[(arc_data['FROM'] == N)&(arc_data['use'] == 1)]['RDWY_LINK_ID'])
    out_N_rteid = list(arc_data[(arc_data['FROM'] == N)&(arc_data['use'] == 1)]['RDWY_RTE_ID'])
    in_N = list(arc_data[(arc_data['TO'] == N)&(arc_data['use'] == 1)]['FROM'])
    in_N_time = list(arc_data[(arc_data['TO'] == N)&(arc_data['use'] == 1)]['TIME'])
    in_N_id = list(arc_data[(arc_data['TO'] == N)&(arc_data['use'] == 1)]['RDWY_LINK_ID'])
    in_N_rteid = list(arc_data[(arc_data['TO'] == N)&(arc_data['use'] == 1)]['RDWY_RTE_ID'])

    new_nodes = list(set(out_N + in_N))
    
    # store a node's corresponding node
    new_nodes_dict = {}
    for i in range(len(new_nodes)):
        new_nodes_dict[new_nodes[i]] = str(N) + "_"+ str(i + 1) 
    
    # connnect new nodes with original nodes
    for index in range(len(out_N)):
        new_arcs.append([new_nodes_dict[out_N[index]],out_N[index],out_N_time[index],out_N_id[index],\
                         out_N_rteid[index],1,0])
        
    for index in range(len(in_N)):
        new_arcs.append([in_N[index],new_nodes_dict[in_N[index]],in_N_time[index],in_N_id[index],\
                         in_N_rteid[index],1,0])
        
    # connect new nodes with each other           
    for i in in_N:
        for o in out_N:
            # do not allow turn around
            if i != o:
                #turn
                if in_N_rteid[in_N.index(i)] != out_N_rteid[out_N.index(o)]:
                    new_arcs.append([new_nodes_dict[i],
                                     new_nodes_dict[o],
                                     -1,
                                     'intersection_'+str(interesction_index),
                                     'intersection_'+str(interesction_index),
                                     1,
                                     (in_N_id[in_N.index(i)],out_N_id[out_N.index(o)])
                                     ])
                #go straight
                else:
                    new_arcs.append([new_nodes_dict[i],new_nodes_dict[o],-2,'intersection_'+str(interesction_index),\
                                 'intersection_'+str(interesction_index),1,0])
                interesction_index += 1
                
    # disconnect original arcs
    arc_data.loc[(arc_data['FROM'] == N) | (arc_data['TO'] == N) , 'use'] = 0
    
    # add arcs to arc set
    new_arcs = pd.DataFrame(new_arcs, columns=["FROM","TO","TIME","RDWY_LINK_ID","RDWY_RTE_ID","use",'turn_rteids'])
    arc_data = arc_data.append(new_arcs,ignore_index=True)
    
# connect alias node according to in and out arcs
    # take note of the original link id and traveling time
    # take note of the length of each link
    
arc_data = arc_data[arc_data['use'] == 1]

arc_data = arc_data.reset_index(drop=True)


In [12]:
arc_data

,FROM,TO,TIME,RDWY_LINK_ID,RDWY_RTE_ID,use,turn_rteids
0,1664328_1,1664328_2,-1.0,intersection_1,intersection_1,1,"(4842901, 4821549)"
1,1664328_2,1664328_1,-1.0,intersection_2,intersection_2,1,"(4842935, 4821515)"
2,1666480_1,1666480_3,-1.0,intersection_3,intersection_3,1,"(4827427, 4827426)"
3,1666480_1,1666480_2,-2.0,intersection_4,intersection_4,1,0
4,1666480_3,1666480_2,-1.0,intersection_5,intersection_5,1,"(4848811, 4827429)"
...,...,...,...,...,...,...,...
9037,1663508_4,1663508_3,-2.0,intersection_5910,intersection_5910,1,0
9038,1663508_4,1663508_2,-1.0,intersection_5911,intersection_5911,1,"(4842054, 4820618)"
9039,1663508_2,1663508_1,-2.0,intersection_5912,intersection_5912,1,0
9040,1663508_2,1663508_3,-1.0,intersection_5913,intersection_5913,1,"(4842004, 4842003)"


In [12]:
arc_data["TRAVEL_TIME"] =  arc_data["TIME"] 
arc_data["TURN"] =  0
arc_data.loc[arc_data['TIME'] < 0,"TRAVEL_TIME"] = 0
arc_data.loc[arc_data['TIME'] == -1,"TURN"] = 1

# in order to conserve data in the gdx file
arc_data["TRAVEL_TIME"] = arc_data["TRAVEL_TIME"] +1

In [13]:
arc_data

,FROM,TO,TIME,RDWY_LINK_ID,RDWY_RTE_ID,use,turn_rteids,TRAVEL_TIME,TURN
0,1664064_2,1664064_3,-2.000000,intersection_1,intersection_1,1,0,1.000000,0
1,1664064_2,1664064_1,-1.000000,intersection_2,intersection_2,1,"(4821275, 4821362)",1.000000,1
2,1664064_3,1664064_1,-1.000000,intersection_3,intersection_3,1,"(4842709, 4821362)",1.000000,1
3,1664064_3,1664064_2,-2.000000,intersection_4,intersection_4,1,0,1.000000,0
4,1664064_1,1664064_3,-1.000000,intersection_5,intersection_5,1,"(4842748, 4821323)",1.000000,1
...,...,...,...,...,...,...,...,...,...
9037,1661974_3,2036956_1,21.199852,5126005,400431,1,0,22.199852,0
9038,1679440_1,1679441_3,5.765192,4884630,405054,1,0,6.765192,0
9039,1679441_3,1679440_1,5.765192,4884629,405054,1,0,6.765192,0
9040,1663054_1,1663076_4,4.240000,4851152,400478,1,0,5.240000,0


# save files

In [42]:
turning_network_time = arc_data[["FROM","TO","TRAVEL_TIME"]]
turning_network_time.to_csv("turning_network_time_updated.csv",index=False, header=False)

In [43]:
turning_network_turn = arc_data[["FROM","TO","TURN"]]
turning_network_turn.to_csv("turning_network_turn_updated.csv",index=False, header=False)

In [45]:
turning_network_linkID = arc_data[["RDWY_LINK_ID","FROM","TO"]]
turning_network_linkID.to_csv("turning_network_linkID_updated.csv",index=False, header=False)


# More information on intersection (From rural to urban)

In [14]:
# read functional class data
class_file = pd.read_csv(os.path.join("transportation_data","wislr-data","wislr_rdwy_link_fnct_cls.csv"))

In [16]:
class_file.head()

,RDWY_LINK_ID,REF_SITE_FROM_ID,REF_SITE_TO_ID,LCM_FROM_TO_DIS,LOC_FLKOS_FT,LOC_TLKOS_FT,ST_PRMY_SYMB_TY,FNCT_CLS_CTGY_TYCD,FNCT_CLS_CTGY_TYDC
0,4818580,1661200,1660820,4066,0,1320,CTH,60.0,Urban Other Connecting Link Principal Arterial
1,4818580,1661200,1660820,4066,1320,2059,CTH,60.0,Urban Other Connecting Link Principal Arterial
2,4818580,1661200,1660820,4066,2059,2746,CTH,60.0,Urban Other Connecting Link Principal Arterial
3,4818580,1661200,1660820,4066,2746,3590,CTH,60.0,Urban Other Connecting Link Principal Arterial
4,4818580,1661200,1660820,4066,3590,4066,CTH,60.0,Urban Other Connecting Link Principal Arterial


In [17]:
#construct 
with open("ruralOrUrban.pk", "rb") as f:
    RUDict = pickle.load(f)

In [20]:
RUDict[4821362]

'Urban'

In [29]:
turn = []
for i in list(arc_data["turn_rteids"]):
    if i == 0:
        turn.append("None")
    else:
        origin = i[0]
        destination = i[1]
        if origin in RUDict and destination in RUDict:
            turn.append((RUDict[origin],RUDict[destination]))
        else:
            turn.append("None")


In [30]:
turn

['None',
 ('Urban', 'Urban'),
 ('Urban', 'Urban'),
 'None',
 ('Urban', 'Urban'),
 ('Urban', 'Urban'),
 ('Urban', 'Urban'),
 'None',
 ('Urban', 'Urban'),
 ('Urban', 'Urban'),
 'None',
 ('Urban', 'Urban'),
 'None',
 ('Rural', 'Rural'),
 ('Rural', 'Rural'),
 'None',
 ('Rural', 'Rural'),
 ('Rural', 'Rural'),
 ('Urban', 'Urban'),
 ('Urban', 'Urban'),
 'None',
 ('Urban', 'Urban'),
 ('Urban', 'Urban'),
 ('Urban', 'Urban'),
 ('Urban', 'Urban'),
 ('Urban', 'Urban'),
 ('Urban', 'Urban'),
 'None',
 ('Urban', 'Urban'),
 ('Urban', 'Urban'),
 'None',
 ('Urban', 'Urban'),
 ('Urban', 'Urban'),
 'None',
 ('Urban', 'Urban'),
 ('Urban', 'Urban'),
 ('Urban', 'Urban'),
 ('Urban', 'Urban'),
 ('Urban', 'Urban'),
 'None',
 ('Rural', 'Urban'),
 'None',
 ('Urban', 'Urban'),
 ('Urban', 'Urban'),
 'None',
 ('Urban', 'Urban'),
 'None',
 ('Urban', 'Urban'),
 ('Urban', 'Urban'),
 ('Urban', 'Urban'),
 'None',
 ('Urban', 'Urban'),
 ('Urban', 'Urban'),
 'None',
 'None',
 ('Urban', 'Urban'),
 'None',
 ('Urban', 'Urban')